In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: ignored

In [ ]:
!pip install transformers
!pip install tqdm

In [2]:
size_name = "base"
reberth_path = "G:\\My Drive\\ANLP21\\\ReBERTh\\base"
data_path = "G:\\My Drive\\ANLP21\\regex_data"
compare_model_dir = "G:\\My Drive\\ANLP21\\ReBERTh\\compare\\base"

In [ ]:
size_name = "base"
reberth_path = f"/content/drive/MyDrive/ANLP21/ReBERTh/{size_name}"
data_path = "/content/drive/MyDrive/ANLP21/regex_data"
compare_model_dir = f"/content/drive/MyDrive/ANLP21/ReBERTh/compare/base"
!mkdir -p $compare_model_dir

In [1]:
!nvidia-smi

Sun Dec 12 12:58:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.77       Driver Version: 466.77       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   33C    P8    15W / 200W |    579MiB /  8192MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import torch.nn as nn
from torch.nn import LSTM,Embedding,Linear
from torch.nn import Module
import torch.nn.functional as F
from torch.autograd import Variable
from transformers import BertModel, BertTokenizerFast, RobertaTokenizerFast, RobertaModel
import tqdm

class compare_regex(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size):
        super(compare_regex, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embed = Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm1 = LSTM(embedding_dim ,hidden_dim, bidirectional=True, num_layers=1, batch_first=True)
        self.lstm2 = LSTM(embedding_dim, hidden_dim, bidirectional=True, num_layers=1, batch_first=True)
        self.fc1 = Linear(hidden_dim*2*2, 60)
        self.fc2 = Linear(60, 20)
        self.fc3 = Linear(20, target_size)

        
    def init_hidden(self, bs):
        if torch.cuda.is_available():
            return (torch.zeros(2, bs, self.hidden_dim).cuda(),
                   torch.zeros(2, bs, self.hidden_dim).cuda())
        else:
            return (torch.zeros(2, bs, self.hidden_dim),
                   torch.zeros(2, bs, self.hidden_dim))
    
    def forward(self, bs, line1, line2, input1_lengths,input2_lengths):
        embeded1 = self.embed(line1)
        embeded2 = self.embed(line2)
        hidden1 = self.init_hidden(bs)
        lstm1_out, last_hidden1 = self.lstm1(embeded1,hidden1)
        hidden2 = self.init_hidden(bs)
        lstm2_out, last_hidden2 = self.lstm1(embeded2,hidden2)
        fc1_out = self.fc1(torch.cat((lstm1_out.mean(1), lstm2_out.mean(1)),1))

        fc1_out = F.tanh(fc1_out)
        fc2_out = self.fc2(fc1_out)
        fc2_out = F.tanh(fc2_out)
        fc3_out = self.fc3(fc2_out)
        score = F.log_softmax(fc3_out,dim=1)
        return score

In [4]:
class compare_regex_bert(torch.nn.Module):
    tokenizer = RobertaTokenizerFast.from_pretrained(compare_model_dir, do_lower_case=False, do_basic_tokenize=False)
    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size):
        super(compare_regex_bert, self).__init__()

        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embed = Embedding(vocab_size, embedding_dim, padding_idx=0)
        # self.lstm1 = LSTM(embedding_dim ,hidden_dim, bidirectional=True, num_layers=1, batch_first=True)
        # self.lstm2 = LSTM(embedding_dim, hidden_dim, bidirectional=True, num_layers=1, batch_first=True)
        # self.model_name=params["model_name"]
        # self.tokenizer = BertTokenizer.from_pretrained("bert-base-cased", do_lower_case=False, do_basic_tokenize=False)
        self.bert1 = RobertaModel.from_pretrained(reberth_path)
        self.bert2 = RobertaModel.from_pretrained(reberth_path)

        self.bert1.resize_token_embeddings(len(self.tokenizer))
        self.bert2.resize_token_embeddings(len(self.tokenizer))

        # self.fc = Linear(embedding_dim*2*2, hidden_dim*2*2)
        self.fc1 = Linear(embedding_dim*2, 60)
        self.fc2 = Linear(60, 20)
        self.fc3 = Linear(20, target_size)

        
    def init_hidden(self, bs):
        if torch.cuda.is_available():
            return (torch.zeros(2, bs, self.hidden_dim).cuda(),
                   torch.zeros(2, bs, self.hidden_dim).cuda())
        else:
            return (torch.zeros(2, bs, self.hidden_dim),
                   torch.zeros(2, bs, self.hidden_dim))
    
    def forward(self, line1, line2):
        # embeded1 = self.embed(line1)
        # embeded2 = self.embed(line2)



        # hidden1 = self.init_hidden(bs)
        # lstm1_out, last_hidden1 = self.lstm1(embeded1,hidden1)
        # hidden2 = self.init_hidden(bs)
        # lstm2_out, last_hidden2 = self.lstm2(embeded2,hidden2)
        # print(line1)
        # print(line2)

        # bert1_output = self.bert1(input_ids=line1["input_ids"],
        #                  attention_mask=line1["attention_mask"],
        #                  token_type_ids=line1["token_type_ids"],
        #                  output_hidden_states=True)
        # bert1_hidden_states = bert1_output['hidden_states']
        # bert1_out = bert1_hidden_states[-1][:,0,:]

        bert1_output = self.bert1(**line1)
        bert1_out = bert1_output.last_hidden_state

        bert2_output = self.bert2(**line2)
        bert2_out = bert2_output.last_hidden_state
        # bert2_output = self.bert2(input_ids=line2["input_ids"],
        #                  attention_mask=line2["attention_mask"],
        #                  token_type_ids=line2["token_type_ids"],
        #                  output_hidden_states=True)
        # bert2_hidden_states = bert2_output['hidden_states']
        # bert2_out = bert2_hidden_states[-1][:,0,:]


        fc1_out = self.fc1(torch.cat((bert1_out.mean(1), bert2_out.mean(1)),1))  #encoder outputs 평균값 concat 97.8%

        
        fc1_out = F.tanh(fc1_out)
        fc2_out = self.fc2(fc1_out)
        fc2_out = F.tanh(fc2_out)
        fc3_out = self.fc3(fc2_out)
        score = F.log_softmax(fc3_out,dim=1)
        return score

======================================================================================

In [5]:
# f = open('/content/drive/MyDrive/ANLP21/train_data.txt','r')
f = open("G:\\My Drive\\ANLP21\\train_data.txt",'r')

total_set = set()
lines1 = list()
lines2 = list()
targets = list()


for line in f.read().splitlines():
    splitted = line.split('\t')
    a = '{}\t{}\t{}'.format(splitted[1],splitted[0],splitted[2])
    if not a in total_set:
        total_set.add(line)
        
train_lines1 = list()
train_lines2 = list()
train_targets = list()
for line in total_set:
    splitted = line.split('\t')
    train_lines1.append(splitted[0])
    train_lines2.append(splitted[1])
    train_targets.append(splitted[2])
    train_lines1.append(splitted[1])
    train_lines2.append(splitted[0])
    train_targets.append(splitted[2])
print(train_lines1[0])

( ( <M0> ) ( . * ) ) & ( \ b [ <VOW> ] \ b )


In [6]:
# f = open('/content/drive/MyDrive/ANLP21/test_data.txt','r')
f = open("G:\\My Drive\\ANLP21\\test_data.txt",'r')

total_set = set()
lines1 = list()
lines2 = list()
targets = list()


for line in f.read().splitlines():
    splitted = line.split('\t')
    a = '{}\t{}\t{}'.format(splitted[1],splitted[0],splitted[2])
    if not a in total_set:
        total_set.add(line)
        
test_lines1 = list()
test_lines2 = list()
test_targets = list()
for line in total_set:
    splitted = line.split('\t')
    test_lines1.append(splitted[0])
    test_lines2.append(splitted[1])
    test_targets.append(splitted[2])

In [7]:
# #delete cell
# f = open('../pair_data/data_pairs_test(4_depth).txt','r')
# # f = open('../pair_data/test_data.txt','r')

# total_set = set()
# lines1 = list()
# lines2 = list()
# targets = list()


# for line in f.read().splitlines():
#     splitted = line.split('\t')
#     a = '{}\t{}\t{}'.format(splitted[1],splitted[0],splitted[2])
#     if not a in total_set:
#         total_set.add(line)
        
# test_lines1 = list()
# test_lines2 = list()
# test_targets = list()
# for line in total_set:
#     splitted = line.split('\t')
#     test_lines1.append(splitted[0])
#     test_lines2.append(splitted[1])
#     test_targets.append(splitted[2])

In [8]:
print(len(train_lines1))
print(len(test_lines1))

372728
5297


==================================================================================================

In [9]:
vocab = {w:i for i,w in enumerate(set([t for s in train_lines1 for t in s.split(' ')]), 1)}
vocab['<pad>'] = 0
vocab_size = len(vocab)
print(len(vocab))

31


In [10]:
print(vocab)

{'b': 1, '[': 2, '<CAP>': 3, '{': 4, '<M3>': 5, '4': 6, '\\': 7, '&': 8, '<NUM>': 9, '<M1>': 10, '2': 11, '<M0>': 12, ']': 13, '|': 14, '<LOW>': 15, '+': 16, '<VOW>': 17, '}': 18, '6': 19, '~': 20, '<LET>': 21, '(': 22, ')': 23, '5': 24, '.': 25, '<M2>': 26, ',': 27, '3': 28, '*': 29, '7': 30, '<pad>': 0}


In [11]:
# f_w = open('./compare_vocab(uncleaned).txt','w')
# for i in vocab.items():
#     f_w.write('{}\t{}\n'.format(i[0],i[1]))
# f_w.close()

In [12]:
vocab = {}
# f = open('/content/drive/MyDrive/ANLP21/compare_vocab.txt','r')
f = open('G:\\My Drive\\ANLP21\\compare_vocab.txt','r')

for i in f.read().splitlines():
    splitted = i.split('\t')
    vocab[splitted[0]] = int(splitted[1])
vocab_size = len(vocab)
compare_regex_bert.tokenizer.add_tokens(list(vocab.keys()))

0

In [13]:
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def make_input_seq(lines1, lines2, targets):
    
    max_len = 40
    lines1_seq2idx = list()
    lines2_seq2idx = list()
    targets_idx = list()
    lines1_seq = [s.split() for s in lines1]
    lines2_seq = [s.split() for s in lines2]
    for line_num in range(len(lines1_seq)):
        if len(lines1_seq[line_num]) > max_len or len(lines2_seq[line_num]) > max_len:
            continue
        # lines1_padded = lines1_seq[line_num]+['<pad>']*(max_len-len(lines1_seq[line_num]))
        # lines2_padded = lines2_seq[line_num]+['<pad>']*(max_len-len(lines2_seq[line_num]))
        # lines1_seq2idx.append([vocab[i] for i in lines1_padded])
        # lines2_seq2idx.append([vocab[i] for i in lines2_padded])
        
        # lines1_tokenized = compare_regex_bert.tokenizer(lines1, padding=True, truncation=True, return_tensors="pt", max_length=256)
        # lines2_tokenized = compare_regex_bert.tokenizer(lines2, padding=True, truncation=True, return_tensors="pt", max_length=256)
        
        if targets[line_num] == '0':
            targets_idx.append([1,0])
        else:
            targets_idx.append([0,1])
    lines1_tokenized = compare_regex_bert.tokenizer(lines1, padding=True, truncation=True, return_tensors="pt", max_length=256)
    lines2_tokenized = compare_regex_bert.tokenizer(lines2, padding=True, truncation=True, return_tensors="pt", max_length=256)
    if torch.cuda.is_available():
        return lines1_tokenized.to(device), lines2_tokenized.to(device), torch.LongTensor(targets_idx).cuda()
    else:
        return lines1_tokenized, lines2_tokenized, torch.LongTensor(targets_idx)
    # return lines1_tokenized, lines2_tokenized, torch.LongTensor(targets_idx)
        


lines1_seq2idx, lines2_seq2idx, targets_idx = make_input_seq(train_lines1, train_lines2, train_targets)
test_input1, test_input2, test_targets = make_input_seq(test_lines1, test_lines2, test_targets)

In [14]:
print(len(lines1_seq2idx))
print(len(lines2_seq2idx))
print(len(test_targets))
print(test_targets.tolist().count([1,0]))

2
2
5297
2500


In [15]:
import torch.optim as optim
import time

In [16]:
def confusion(prediction, truth):
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

In [17]:
from transformers import tokenization_utils_base

batch_size = 64
def evaluate_test(model, test_input1, test_input2 , test_target):
    correct = 0
    print(len(test_target))
    tp=0
    tn=0
    fp=0
    fn=0
    batch_num = int(len(test_lines1)/batch_size)
    for batch in range(batch_num):
        # test_input1_len = torch.tensor([torch.max(test_input1[i].data.nonzero()+1)])
        # test_input2_len = torch.tensor([torch.max(test_input2[i].data.nonzero()+1)])
        # score = model(1, test_input1[i].unsqueeze(0), test_input2[i].unsqueeze(0) , test_input1_len.tolist(), test_input2_len.tolist())
        lines1_batch = {k: v[batch * batch_size:(batch+1) * batch_size] for k, v in test_input1.items()}
        lines2_batch = {k: v[batch * batch_size:(batch+1) * batch_size] for k, v in test_input2.items()}
        targets_batches = targets_idx[batch * batch_size:(batch+1) * batch_size].to(device)
        score = model(lines1_batch, lines2_batch)

        true_pos, false_pos, true_neg, false_neg = confusion(score.argmax(dim=1), targets_batches[:, 1])
        tp += true_pos
        fp += false_pos
        tn += true_neg
        fn += false_neg
        # print(score.argmax(dim=1))
    #     if score.argmax().item() == 1 and test_target[i].argmax().item()==1:
    #         tp+=1
    #     elif score.argmax().item() == 0 and test_target[i].argmax().item()==0:
    #         tn+=1
    #     elif score.argmax().item() == 1 and test_target[i].argmax().item()==0:
    #         fp+=1
    #     elif score.argmax().item() == 0 and test_target[i].argmax().item()==1:
    #         fn+=1
    #     if score.argmax().item() == test_target[i].argmax().item():
    #         correct += 1
    try:
        precision =  tp/(tp+fp)
        recall =  tp/(tp+fn)
        f1_score = 2*((precision*recall)/(precision+recall))
        correct = tp + tn
    except:
        precision = 0
        recall = 0
        f1_score = 0
    print('precision: {},recall: {},f1 score:{}'.format(precision,recall,f1_score))
    print('total: {}, correct: {}'.format(len(test_target), correct))
    return correct/len(test_target)

In [19]:
import os
print(torch.cuda.is_available())
if torch.cuda.is_available():
    # compare_regex_model = compare_regex_bert(vocab_size, 256, 256, 2).cuda()
    # compare_regex_model = compare_regex_bert(vocab_size, 768, 256, 2).cuda()
    compare_regex_model = torch.load(os.path.join(compare_model_dir, "epoch-1-batch-1000.pth"))
else:
    compare_regex_model = compare_regex_bert(vocab_size, 768, 256, 2)
    # compare_regex_model = compare_regex_bert(vocab_size, 256, 256, 2)

True


In [23]:
torch.cuda.empty_cache()

In [24]:
!nvidia-smi

Sun Dec 12 13:05:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.77       Driver Version: 466.77       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P8    15W / 200W |   8093MiB /  8192MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(compare_regex_model.parameters(), lr=0.1)
batch_size = 64

start_epoch, start_batch = 1, 1000

batch_num = int(len(train_lines1)/batch_size)
print("num_batches:", batch_num)

compare_regex_model.train()
for epoch in range(start_epoch, 5):
    epoch_loss = 0
    start_time = time.time()
    if epoch == start_epoch and start_batch != 0:
      for batch in tqdm.tqdm(range(start_batch+1, batch_num)):
        compare_regex_model.zero_grad()
        # lines1_batch = lines1_seq2idx[batch * batch_size:(batch+1) * batch_size]
        # lines2_batch = lines2_seq2idx[batch * batch_size:(batch+1) * batch_size]
        lines1_batch = {k: v[batch * batch_size:(batch+1) * batch_size] for k, v in lines1_seq2idx.items()}
        lines2_batch = {k: v[batch * batch_size:(batch+1) * batch_size] for k, v in lines2_seq2idx.items()}
        # lines1_batch_lengths = torch.tensor([torch.max(lines1_batch[i].data.nonzero()+1) for i in range(len(lines1_batch))]).cuda()
        # lines2_batch_lengths = torch.tensor([torch.max(lines2_batch[i].data.nonzero()+1) for i in range(len(lines2_batch))]).cuda()
        tag_score = compare_regex_model(lines1_batch, lines2_batch)
        targets_batches = targets_idx[batch * batch_size:(batch+1) * batch_size]
        # targets_batches = targets_idx
        # print(tag_score.squeeze(1), targets_batches)
        loss = loss_function(tag_score.squeeze(1).squeeze(1), targets_batches[:, 1])
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch > 0 and batch % 1000 == 0:
          torch.save(compare_regex_model, os.path.join(compare_model_dir, f"epoch-{epoch}-batch-{batch}.pth"))
          compare_regex_model.tokenizer.save_pretrained(compare_model_dir)
        # if batch % 100 == 0:
        #   print('iter: {}, batch_loss: {}'.format(batch, loss.item()))
    else:
      for batch in tqdm.tqdm(range(batch_num)):
          compare_regex_model.zero_grad()
          # lines1_batch = lines1_seq2idx[batch * batch_size:(batch+1) * batch_size]
          # lines2_batch = lines2_seq2idx[batch * batch_size:(batch+1) * batch_size]
          lines1_batch = {k: v[batch * batch_size:(batch+1) * batch_size] for k, v in lines1_seq2idx.items()}
          lines2_batch = {k: v[batch * batch_size:(batch+1) * batch_size] for k, v in lines2_seq2idx.items()}
          # lines1_batch_lengths = torch.tensor([torch.max(lines1_batch[i].data.nonzero()+1) for i in range(len(lines1_batch))]).cuda()
          # lines2_batch_lengths = torch.tensor([torch.max(lines2_batch[i].data.nonzero()+1) for i in range(len(lines2_batch))]).cuda()
          tag_score = compare_regex_model(lines1_batch, lines2_batch)
          targets_batches = targets_idx[batch * batch_size:(batch+1) * batch_size]
          # targets_batches = targets_idx
          # print(tag_score.squeeze(1), targets_batches)
          loss = loss_function(tag_score.squeeze(1).squeeze(1), targets_batches[:, 1])
          epoch_loss += loss.item()
          loss.backward()
          optimizer.step()
          if batch > 0 and batch % 1000 == 0:
            torch.save(compare_regex_model, os.path.join(compare_model_dir, f"epoch-{epoch}-batch-{batch}.pth"))
            compare_regex_model.tokenizer.save_pretrained(compare_model_dir)
          # if batch % 100 == 0:
          #   print('iter: {}, batch_loss: {}'.format(batch, loss.item()))
    
    with torch.no_grad():
        test_acc = evaluate_test(compare_regex_model, test_input1, test_input2, test_targets)
        print('step:{}, test acc: {}'.format(epoch, test_acc))
    
    if test_acc == 1.0:
        break
    print('epoch: {}, epoch_loss: {}'.format(epoch,epoch_loss/batch_num))
    torch.save(compare_regex_model, os.path.join(compare_model_dir, 'epoch-{}.pth'.format(epoch)))
    compare_regex_model.tokenizer.save_pretrained(compare_model_dir)


num_batches: 5823


  0%|                                                                                         | 0/4822 [00:08<?, ?it/s]


RuntimeError: ignored

In [ ]:
batch_num = int(len(lines1_seq2idx)/batch_size)

with torch.no_grad():
    test_acc = evaluate_test(compare_regex_model, test_input1, test_input2, test_targets)
    print('test acc: {}'.format(test_acc))
    
    print('epoch: {}, epoch_loss: {}'.format(epoch,epoch_loss/batch_num))

In [ ]:
torch.save(compare_regex_model, './compare_regex_model_share2.pth')

In [ ]:
import os
compare_regex_model = torch.load(os.path.join(compare_model_dir, 'epoch-{}.pth'.format(0)), map_location=torch.device('cpu'))

### single input test

In [ ]:
gold = ['( ( <M0> ) & ( [ <LET> ] ) ) . * ( [ <CAP> ] ) . *']
predict = ['( <M0> ) . * ( ( [ <CAP> ] ) & ( [ <CAP> ] ) ) . *']
target = [0]
gold_input, predict_input, target_input = make_input_seq(gold, predict, target)
# gold_len = torch.tensor([torch.max(gold_input[0].data.nonzero()+1)])
# predict_len = torch.tensor([torch.max(predict_input[0].data.nonzero()+1)])
print(predict_input)

{'input_ids': tensor([[   0,   12,  225, 1000,  225,   13,  225,   18,  225,   14,  225,   12,
          225,   12,  225,   63,  225, 1008,  225,   65,  225,   13,  225,   10,
          225,   12,  225,   63,  225, 1008,  225,   65,  225,   13,  225,   13,
          225,   18,  225,   14,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
compare_regex_model(gold_input, predict_input)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[-0.6959, -0.6904]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
evaluate_test(compare_regex_model, test_input1, test_input2, test_targets)

5297


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


precision: 0.37724935732647813,recall: 0.8884964682139254,f1 score:0.529624060150376
total: 5297, correct: 0


0.0

In [ ]:
snew_vocab = dict(map(reversed, vocab.items()))
print(new_vocab)

{1: '', 2: '<M0>', 13: 'b', 3: '[', 14: '(', 4: ',', 6: '<M3>', 16: '.', 17: '\\', 18: '<M1>', 19: '*', 12: '<LOW>', 5: '{', 21: '}', 22: '<M2>', 24: ']', 0: '<pad>', 25: '|', 7: '+', 8: '6', 27: '<LET>', 26: '&', 28: '<VOW>', 23: '<NUM>', 9: '4', 29: '~', 31: '7', 10: '3', 11: ')', 15: '5', 30: '<CAP>', 20: '2'}


In [ ]:
' '.join([new_vocab[j] for j in [i for i in gold_input.tolist()[0]]]).replace('<pad>','')


AttributeError: ignored

In [ ]:
import math
with torch.no_grad():
    score = compare_regex_model(1, gold_input, predict_input, [gold_len], [predict_len])
    print(math.exp(score[0][0]))
    print(math.exp(score[0][1]))


In [ ]:
f = open('../pair_data/test.txt','r')


total_set = set()
lines1 = list()
lines2 = list()
targets = list()

count = 0
for line in f.read().splitlines():
    count += 1
    total_set.add(line)
    splitted = line.split('\t')
#     total_set.add('{}\t{}\t{}'.format(splitted[1],splitted[0],splitted[2]))
print(count)

count = 0
for line in total_set:
    count += 1
    splitted = line.split('\t')
    lines1.append(splitted[0])
    lines2.append(splitted[1])
    targets.append(splitted[2])


In [ ]:
test_input1, test_input2, test_targets = make_input_seq(lines1, lines2, targets)

In [ ]:
with torch.no_grad():
        print('test acc: {}'.format(evaluate_test(compare_regex_model, test_input1, test_input2,  test_targets)))

In [ ]:
compare_regex_model

compare_regex_bert(
  (embed): Embedding(32, 768, padding_idx=0)
  (bert1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(1009, 768)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
        